In [1]:
import pandas as pd
import json
import datetime
import requests
import numpy as np

### List of cities and Ticketmaster codes
City_List = 
    'Denver' : 264,
    'San Francisco Bay': 382,
    'Portland': 362,
    'Los Angeles': 27,
    'Las Vegas' : 14,
    'Phoenix': 36, 
    'Seattle': 42,
    'Austin' : 40, 
    'Houston': 22,
    'Dallas' : 5,
    'Chicago': 3,
    'Nashville': 31,
    'Atlanta': 10,
    'Boston' : 11,
    'New York': 35,
    'Washington DC': 47,
    'Miami' : 15
### Ticketmaster API info    
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&apikey={apikey}'
api_key = 'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'
dmaId = [382,362,264]
marketId =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]



### Define Functions

Define functions for pulling event data from Ticketmaster API and formatting dataframe

In [2]:
#DMA and Market numbers
dma_list = [382,362,264]
mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(dma=None,market=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': 'ticketmaster,frontgate,ticketweb',
               'classificationName': 'music',
               'size': '200',
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj['page']['totalPages']

#Get TicketMaster data, return a dataframe
def getTicketMasterData(dma=None,market=None,page=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': 'ticketmaster,frontgate',
               'classificationName' : 'music',
               'sort': 'date,name,asc',
               'size': '200',
               'page': page,
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'TM_id': event.get('id',{}),
            'TM_name' : event.get('name',{}),
            'TM_artist': list(attraction.get('name') for attraction in event['_embedded'].get('attractions',{})),
            'TM_venue' : list(venue.get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_city' : list(venue.get('city',{}).get('name') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_venue_state' : list(venue.get('state',{}).get('stateCode') for venue in event['_embedded'].get('venues',{}))[0],
            'TM_description' : event.get('description',{}),
            'TM_more_info' : event.get('additionalInfo',{}),
            'TM_start_date' : event.get('dates',{}).get('start',{}).get('dateTime',{}),
            'TM_timezone' : event.get('dates',{}).get('timezone'),
            'TM_span_multiple_days' : event.get('dates',{}).get('spanMultipleDays'),
            'TM_presale_date_start' : list(presale.get('startDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_presale_date_end' : list(presale.get('endDateTime',{}) for presale in event.get('sales').get('presales',{})),
            'TM_sale_date_start' : event.get('sales',{}).get('public',{}).get('startDateTime'),
            'TM_FV_prices': event.get('priceRanges'),
            'TM_promoter': event.get('promoter',{}).get('name'),
            'TM_genre' : event.get('classifications'),
            'TM_place' : event.get('place')
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

def convert_times(df,times_list,tz_col):
    #Loop through each timezone
    df_list = []
    for tz in df[tz_col].unique():
        #Filter rows by timezone
        mask = (df[tz_col] == tz)
        df_local = df.loc[mask]
        #Loop through each datetime row
        for col in times_list:
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
            df_local[col] = pd.to_datetime(df_local[col],errors='coerce').dt.tz_localize('UTC').dt.tz_convert(tz)
            #Convert each column to datetime series, localize to UTC and then convert to proper timezone
        df_list.append(df_local)
    df = pd.concat(df_list, axis=0)
    return df
        
#Get SeatGeek data

### Pull the data and create the data frame
For each city/market, get the number of pages of data, pull all data, and then concatanate the dataframes

In [3]:
fv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_dict = dict()
mkt_dict = dict()
#Get number of pages for face value dma items
for dma in dma_list:
    dma_dict[dma] = get_number_of_TM_pages(dma=dma)
    
#Get DMA data
for dma in dma_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page))
    
#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict[mkt] = get_number_of_TM_pages(market=mkt)
    
#Get market data
for mkt in mkt_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page))
        
#Combine all dataframes for full data on events and face_value prices
fv_df = pd.concat(fv_df_list,axis=0)

fv_df.head()

,TM_FV_prices,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,TM_sale_date_start,TM_span_multiple_days,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state
0,None,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1ae0Zfg0EfsZdGG7,{},"Before the Brave, Billie Gayle",None,[],[],None,2017-10-25T20:40:01Z,False,2017-12-14T04:00:00Z,America/Los_Angeles,Brick and Mortar Music Hall,San Francisco,CA
1,"[{'type': 'standard', 'currency': 'USD', 'min'...","[Miguel, TK Kravitz]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfge8E3we,{},KMEL Holiday House of Soul: Miguel,None,"[2017-10-20T06:59:00Z, 2017-10-20T06:59:00Z]","[2017-10-19T17:00:00Z, 2017-10-19T17:00:00Z]",ANOTHER PLANET,2017-10-20T17:00:00Z,False,2017-12-14T04:00:00Z,America/Los_Angeles,Fox Theater - Oakland,Oakland,CA
2,None,[Sol Horizon],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfgGkBKnpzm,{},Sol Horizon,None,[],[],None,2017-10-31T18:00:01Z,False,2017-12-14T05:30:00Z,America/Los_Angeles,Blue Note Napa,Napa,CA
3,None,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfgGkBbppy1,{},"Kellie Fuller, Sandy Riccardi & Mike Greensill...",None,[],[],None,2017-10-31T18:00:01Z,False,2017-12-15T03:30:00Z,America/Los_Angeles,Blue Note Napa,Napa,CA
4,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Chris Robinson Brotherhood],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfwhgLEYd,{},Chris Robinson Brotherhood,None,[],[],LIVE NATION MUSIC,2017-09-15T17:00:00Z,False,2017-12-15T04:00:00Z,America/Los_Angeles,The Fillmore,San Francisco,CA


## Convert datetime columns to datetime objects in  proper timezones
Currently the four columns with datetime information are a mess. Some are of type dict, some are nested in lists, and all are in the UTC timezone. We can look and see that in timedate columns with lists of dates, the timedates are identitical, so we can unnest datetimes by simply taking the first element of the list.

The 3 things we need to accomplish:
    1. Convert all datetime columns to dtype string, and unnest 'TM_presale_date_end' and 'TM_presale_date_start' columns
    2. Convert all columns to datetime series localized to UTC (Done in the convert_times function)
    3. Filter rows by timezone, and loop over each datetime column to convert objects to their proper timezone (Done in the convert_times function)

In [4]:
# 1. Convert all datetime columns to dtype string, and unnest objects
fv_df['TM_presale_date_end'] = fv_df['TM_presale_date_end'].str[0]
fv_df['TM_presale_date_start'] = fv_df['TM_presale_date_start'].str[0]
fv_df['TM_start_date'] = fv_df['TM_start_date'].astype(str)
fv_df['TM_sale_date_start'] = fv_df['TM_sale_date_start'].astype(str)


In [5]:
# 2. Convert all columns to datetime series localized to UTC
# 3. Filter rows by timezone and create a new dataframe per timezone, loop over each datetime column to convert objects to their proper timezone 
col_list = ['TM_presale_date_end','TM_presale_date_start','TM_start_date','TM_sale_date_start']
fv_df = convert_times(df=fv_df,times_list=col_list,tz_col='TM_timezone')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Explode nested rows
Unnest information in prices and genre column

In [251]:
def explode(df,col,index):
    df1 = df[col].apply(pd.Series)
    df1.index = df[index]
    df1 = df1[0].apply(pd.Series)
    df = df.join(df1, on=index)
    df = df.drop(col,axis=1)
    return df
#Explode price dictionaries
fv_df = explode(fv_df,'TM_FV_prices','TM_id')

/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/usr/local/lib/python3.6/site-packages/pandas/core/indexes/api.py:57: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)


,TM_artist,TM_description,TM_genre,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,...,TM_start_date,TM_timezone,TM_venue,TM_venue_city,TM_venue_state,0,currency,max,min,type
0,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1ae0Zfg0EfsZdGG7,{},"Before the Brave, Billie Gayle",None,NaT,NaT,None,...,2017-12-13 20:00:00-08:00,America/Los_Angeles,Brick and Mortar Music Hall,San Francisco,CA,NaN,NaN,NaN,NaN,NaN
1,"[Miguel, TK Kravitz]",{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfge8E3we,{},KMEL Holiday House of Soul: Miguel,None,2017-10-19 23:59:00-07:00,2017-10-19 10:00:00-07:00,ANOTHER PLANET,...,2017-12-13 20:00:00-08:00,America/Los_Angeles,Fox Theater - Oakland,Oakland,CA,NaN,USD,69.95,39.95,standard
2,[Sol Horizon],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfgGkBKnpzm,{},Sol Horizon,None,NaT,NaT,None,...,2017-12-13 21:30:00-08:00,America/Los_Angeles,Blue Note Napa,Napa,CA,NaN,NaN,NaN,NaN,NaN
3,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfgGkBbppy1,{},"Kellie Fuller, Sandy Riccardi & Mike Greensill...",None,NaT,NaT,None,...,2017-12-14 19:30:00-08:00,America/Los_Angeles,Blue Note Napa,Napa,CA,NaN,NaN,NaN,NaN,NaN
4,[Chris Robinson Brotherhood],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfwhgLEYd,{},Chris Robinson Brotherhood,None,NaT,NaT,LIVE NATION MUSIC,...,2017-12-14 20:00:00-08:00,America/Los_Angeles,The Fillmore,San Francisco,CA,NaN,USD,32.50,32.50,standard


In [277]:
#Unnest genre info and clean up data in exploded columns
fv_df = explode(fv_df,'TM_genre','TM_id')
for col in ['segment','subGenre','subType','type']:
    fv_df.col = fv_df.col.map(lambda x: x.get('name',{}))
fv_df.head()

,TM_FV_prices,TM_artist,TM_description,TM_id,TM_more_info,TM_name,TM_place,TM_presale_date_end,TM_presale_date_start,TM_promoter,...,TM_timezone,TM_venue,TM_venue_city,TM_venue_state,genre,primary,segment,subGenre,subType,type
0,None,[],{},1ae0Zfg0EfsZdGG7,{},"Before the Brave, Billie Gayle",None,NaT,NaT,None,...,America/Los_Angeles,Brick and Mortar Music Hall,San Francisco,CA,Folk,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Folk,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}"
1,"[{'type': 'standard', 'currency': 'USD', 'min'...","[Miguel, TK Kravitz]",{},G5vYZfge8E3we,{},KMEL Holiday House of Soul: Miguel,None,2017-10-19 23:59:00-07:00,2017-10-19 10:00:00-07:00,ANOTHER PLANET,...,America/Los_Angeles,Fox Theater - Oakland,Oakland,CA,R&B,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",R&B,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}"
2,None,[Sol Horizon],{},1Ae0ZfgGkBKnpzm,{},Sol Horizon,None,NaT,NaT,None,...,America/Los_Angeles,Blue Note Napa,Napa,CA,Reggae,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Reggae,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}"
3,None,[],{},1Ae0ZfgGkBbppy1,{},"Kellie Fuller, Sandy Riccardi & Mike Greensill...",None,NaT,NaT,None,...,America/Los_Angeles,Blue Note Napa,Napa,CA,Jazz,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Jazz,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}"
4,"[{'type': 'standard', 'currency': 'USD', 'min'...",[Chris Robinson Brotherhood],{},G5vYZfwhgLEYd,{},Chris Robinson Brotherhood,None,NaT,NaT,LIVE NATION MUSIC,...,America/Los_Angeles,The Fillmore,San Francisco,CA,Rock,True,"{'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'}",Alternative Rock,"{'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'}","{'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'}"
